In [1]:
import geemap
import ee
import os
import calendar
import os
from tolocal import export_image
Map = geemap.Map()
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [ ]:
ndvi_collections = ee.ImageCollection("NOAA/CDR/AVHRR/NDVI/V5") 
Map.addLayer(ndvi_collections, {}, 'ndvi_collections')

word_shp = "G:/图层/word_shp/word_polygon.shp"
roi = geemap.shp_to_ee(word_shp).geometry()

Map.addLayer(roi, {}, 'word')

In [ ]:
import datetime
def get_date_list(start_date, end_date, format,step):
    """
    根据开始日期、结束日期返回这段时间里所有天的集合
    :param start_date: 开始日期(日期格式或者字符串格式)
    :param end_date: 结束日期(日期格式或者字符串格式)
    :param format: 格式化字符串, 如: '%Y-%m-%d'
    :param step: int, 如: (1,8,30)
    :return:
    """
    date_list = []
    if isinstance(start_date, str) and isinstance(end_date, str):
        start_date = datetime.datetime.strptime(start_date, '%Y-%m-%d')
        end_date = datetime.datetime.strptime(end_date, '%Y-%m-%d')
    date_list.append(start_date.strftime(format))
    while start_date < end_date:
        start_date += datetime.timedelta(days=step)
        date_list.append(start_date.strftime(format))
    return date_list

In [ ]:
def clip_dow_merge(geo: ee.Geometry, image: ee.Image, outfile: str, scale: int,
                   crs='epsg:4326', sep=0.25):
    """

    Args:
        geo: ee.Geometry, 需要下载的区域矢量几何
        image: ee.Image, 需要下载的影像
        outfile: str, 输出文件路径和名称，不需要文件后缀，下载的影响默认后缀为tif
        scale: int, 下载时的像元大小
        crs: str, 下载影像的投影，默认为 'epsg:4326' wgs1984投影
        sep: float, 单波段10m分辨率像元的影像裁剪大小(单位：经纬度)，默认为0.25
    Returns: None

    """
    import os
    import numpy as np
    import rasterio
    from glob import glob
    from rasterio.merge import merge
    import shutil
    import geemap
    import math
    bounds = geo.bounds()
    bands = image.bandNames().size().getInfo()
    poy = np.array(bounds.coordinates().getInfo()[0])
    min_x = poy[:, 0].min()
    max_x = poy[:, 0].max()
    min_y = poy[:, 1].min()
    max_y = poy[:, 1].max()
    step = scale / 10 * sep / (int(math.sqrt(bands))+1)
    end_x = int((max_x - min_x) / step) + 1
    end_y = int((max_y - min_y) / step) + 1
    polys = []
    for i in range(end_y):
        y1 = min_y + step * i
        y2 = min_y + step * (i + 1)
        if y2 > max_y:
            y2 = max_y
        for j in range(end_x):
            x1 = min_x + step * j
            x2 = min_x + step * (j + 1)
            if x2 > max_x:
                x2 = max_x
            poly = ee.Geometry(ee.Geometry.Rectangle([float(x1), float(y1), float(x2), float(y2)]), None, False)
            polys.append(poly)
    if len(polys) > 1:
        print(f"分割成{len(polys)}份, 开始下载:")
        path = outfile+'_mk'
        t = 1
        if not os.path.exists(path):
            t = 0
            os.makedirs(path)
        for j, i in enumerate(polys):
            if not os.path.exists(path+f'/temp_{j}.tif'):
                if t:
                    clip_dow_merge(i, image, path + f'/temp_{j}', scale, sep=sep*0.5)
                else:
                    geemap.ee_export_image(image, path + f'/temp_{j}.tif', scale=scale, crs=crs, region=i)
            else:
                continue
        files = glob(path+"/*.tif")
        if len(files) == len(polys):
            src_files_to_mosaic = []
            for tif_f in files:
                src = rasterio.open(tif_f)
                src_files_to_mosaic.append(src)
            mosaic, out_trans = merge(src_files_to_mosaic)
            out_meta = src.meta.copy()
            out_meta.update({"driver": "GTiff",
                             "height": mosaic.shape[1],
                             "width": mosaic.shape[2],
                             "transform": out_trans,
                             })
            with rasterio.open(outfile+".tif", "w", **out_meta) as dest:
                dest.write(mosaic)
            for src in src_files_to_mosaic:
                src.close()
            shutil.rmtree(path)
        else:
            print('下载不完全！！')
    else:
        geemap.ee_export_image(image, outfile+'.tif', scale=scale, crs=crs, region=geo)
    print("download successful !!!")

In [ ]:
for year in range(1982,2019):
    start_year = str(year) + '-01-01'
    end_year = str(year+1) + '-01-01'
    year_list = get_date_list(start_year,end_year,'%Y-%m-%d',8)
    print(year_list)
    for name ,data in enumerate(year_list[:-1]):
        start_data = year_list[name]
        end_data = year_list[name+ 1]
        ndvi_image_8day = ndvi_collections.filterDate(start_data,end_data).select('NDVI')
        print(f'{start_data}-{end_data}:{ndvi_image_8day.size().getInfo()}')
        if ndvi_image_8day.size().getInfo()==0:
            print('ndvi_image_8days is 0, continue---------')
            continue
        ndvi_8day_mean = ndvi_image_8day.mean().clip(roi)
        filename = r'F:\W_Data\NDVI' + os.sep + str(year) + start_data + '.tif'
        print('filename:  ',filename)
#         out_name = 'F:/W_Data/NDVI'+ os.sep + str(year) + start_data 
#         while not os.path.exists(out_name+'.tif'):
#             clip_dow_merge(roi, ndvi_8day_mean, out_name, int(round(ndvi_8day_mean.projection().nominalScale().getInfo())))
        try:
            geemap.ee_export_image(ndvi_8day_mean, filename = filename, scale=int(round(ndvi_8day_mean.projection().nominalScale().getInfo())),region=roi,file_per_band=False)
        except:
            geemap.ee_export_image(ndvi_8day_mean, filename = filename, scale=int(round(ndvi_8day_mean.projection().nominalScale().getInfo())),region=roi,file_per_band=False)
        print(f'{filename} is ok ')
    print(f'{year} is ok ')
print('---------all is ok---------------')